In [1]:
import pandas as pd
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 500)

import numpy as np
import os
import re

import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('dark_background')

from tqdm.notebook import tqdm

In [2]:
wDir = os.path.join('C:\\','Users','eugen','Dropbox','Upenn','projects','COVID')
inDir = os.path.join(wDir,'data','input')
outDir = os.path.join(wDir,'data','output')

In [3]:
# We upload the list of FIPS codes that need to be changed and convert it to a dictionary
FIPSFixes = pd.read_csv(os.path.join(inDir,'FIPSHarmonization','FIPSFixes.csv'),dtype=str)
FIPSFixes = FIPSFixes.set_index('FIPSCode').to_dict()

## Intercensal Population Estimates 1990-2000

In [4]:
popDataPath = os.path.join(inDir,'popData','popInterCensEst19902000.txt')
popCols = [f'pop{year}' for year in range(2000,1989,-1)]
newPopCols = [f'pop{year}' for year in range(1990,2000)]
colNames = ['block','FIPSCode'] + popCols + ['popApril1990','areaName']

dtypes = {'B':str, 'County':str, '7/1/2000':str, '7/1/1999':str, '7/1/1998':str, '7/1/1997':str,
          '7/1/1996':str, '7/1/1995':str, '7/1/1994':str, '7/1/1993':str, '7/1/1992':str, 
          '7/1/1991':str, '7/1/1990':str, '4/1/1990':str, 'Area Name':str}

popData1990 = pd.read_fwf(popDataPath, converters=dtypes)

In [5]:
popData1990.columns = colNames
popData1990 = popData1990[['FIPSCode'] + newPopCols]
popData1990 = popData1990.dropna()
popData1990 = popData1990[popData1990.FIPSCode.apply(lambda x: len(x) == 5)]
popData1990[newPopCols] = popData1990[newPopCols].applymap(lambda x: int(x.replace(',','')))

In [6]:
# We drop state-level estimates
popData1990 = popData1990[~(popData1990.FIPSCode.apply(lambda x: x.endswith('000')))]

In [7]:
# Correct county FIPS that changed over time
popData1990 = popData1990.replace(to_replace = {'FIPSCode':FIPSFixes['newFIPSCode']})
popData1990 = popData1990.groupby('FIPSCode').sum().reset_index()

## Intercensal Population Estimates 2000-2010

In [8]:
cols = ['STATE','COUNTY','STNAME','CTYNAME'] + [f'POPESTIMATE{year}' for year in range(2000,2010)]

popData2000 = pd.read_csv(os.path.join(inDir,'popData','popInterCensEst20002010.csv'),encoding="ISO-8859-1",usecols=cols)

In [9]:
def zero_padding(number,size=1):
    return f"{'0'*(len(str(10**size)) - len(str(int(number))))}{int(number)}"

In [10]:
popData2000['FIPSCode'] = popData2000[['STATE','COUNTY']].apply(lambda x: zero_padding(x.STATE*1000 + x.COUNTY,size=4),axis=1)

In [11]:
popData2000 = popData2000.rename(columns={f'POPESTIMATE{year}':f'pop{year}' for year in range(2000,2010)})
popData2000 = popData2000[['FIPSCode'] + [f'pop{year}' for year in range(2000,2010)]]

In [12]:
# We drop state-level estimates
popData2000 = popData2000[~(popData2000.FIPSCode.apply(lambda x: x.endswith('000')))]

In [13]:
# Correct county FIPS that changed over time
popData2000 = popData2000.replace(to_replace = {'FIPSCode':FIPSFixes['newFIPSCode']})
popData2000 = popData2000.groupby('FIPSCode').sum().reset_index()

## Intercensal Population Estimates 2010-2020

In [14]:
cols = ['STATE','COUNTY','STNAME','CTYNAME'] + [f'POPESTIMATE{year}' for year in range(2010,2020)]

popData2010 = pd.read_csv(os.path.join(inDir,'popData','popInterCensEst20102020.csv'),encoding="ISO-8859-1", usecols=cols)

In [15]:
popData2010['FIPSCode'] = popData2010[['STATE','COUNTY']].apply(lambda x: zero_padding(x.STATE*1000 + x.COUNTY,size=4),axis=1)

In [16]:
popData2010 = popData2010.rename(columns={f'POPESTIMATE{year}':f'pop{year}' for year in range(2010,2020)})
popData2010 = popData2010[['FIPSCode'] + [f'pop{year}' for year in range(2010,2020)]]

In [17]:
# We drop state-level estimates
popData2010 = popData2010[~(popData2010.FIPSCode.apply(lambda x: x.endswith('000')))]

In [18]:
# Correct county FIPS that changed over time
popData2010 = popData2010.replace(to_replace = {'FIPSCode':FIPSFixes['newFIPSCode']})
popData2010 = popData2010.groupby('FIPSCode').sum().reset_index()

## Population Estimates 2020-2021

In [19]:
cols = ['STATE','COUNTY','STNAME','CTYNAME'] + [f'POPESTIMATE{year}' for year in range(2020,2022)]

popData2020 = pd.read_csv(os.path.join(inDir,'popData','popEst20202021.csv'),encoding="ISO-8859-1", usecols=cols)

In [20]:
popData2020['FIPSCode'] = popData2020[['STATE','COUNTY']].apply(lambda x: zero_padding(x.STATE*1000 + x.COUNTY,size=4),axis=1)

In [21]:
popData2020 = popData2020.rename(columns={f'POPESTIMATE{year}':f'pop{year}' for year in range(2020,2022)})
popData2020 = popData2020[['FIPSCode'] + [f'pop{year}' for year in range(2020,2022)]]

In [22]:
# We drop state-level estimates
popData2020 = popData2020[~(popData2020.FIPSCode.apply(lambda x: x.endswith('000')))]

In [23]:
# Correct county FIPS that changed over time
popData2020 = popData2020.replace(to_replace = {'FIPSCode':FIPSFixes['newFIPSCode']})
popData2020 = popData2020.groupby('FIPSCode').sum().reset_index()

## Bringing All Together

In [24]:
popDataWide = popData1990.merge(popData2000,on='FIPSCode',how='outer')
popDataWide = popDataWide.merge(popData2010,on='FIPSCode',how='outer')
popDataWide = popDataWide.merge(popData2020,on='FIPSCode',how='outer')

In [25]:
# NOTE: All issues with the FIPS codes should be solved now

In [26]:
popData = pd.wide_to_long(popDataWide,stubnames='pop',i='FIPSCode',j='year').reset_index()

In [27]:
popData = popData.sort_values(by=['FIPSCode','year']).reset_index(drop=True)

In [28]:
popData['popLead'] = popData.groupby('FIPSCode')['pop'].shift(-1)
popData['popLead'] = popData.popLead.where(~np.isnan(popData.popLead),popData['pop'])

In [29]:
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=2)
popData['avgPop'] = popData.groupby('FIPSCode')['pop'].rolling(window=indexer,min_periods=1).mean().reset_index()['pop']

In [30]:
monthSeq = [7,8,9,10,11,12,1,2,3,4,5,6]

months1990 = pd.Series([month for FIPSCode in popData.FIPSCode.unique() for month in range(7,13)])
months2021 = pd.Series([month for FIPSCode in popData.FIPSCode.unique() for month in range(1,7)])
otherMonths = pd.Series([month for FIPSCode in popData.FIPSCode.unique() for year in range(1991,2021) for month in range(1,13)])

years1990 = pd.Series([1990 for FIPSCode in popData.FIPSCode.unique() for month in range(7,13)])
years2021 = pd.Series([2021 for FIPSCode in popData.FIPSCode.unique() for month in range(1,7)])
otherYears = pd.Series([year for FIPSCode in popData.FIPSCode.unique() for year in range(1991,2021) for month in range(1,13)])

FIPS1990 = pd.Series([FIPSCode for FIPSCode in popData.FIPSCode.unique() for month in range(7,13)])
FIPS2021 = pd.Series([FIPSCode for FIPSCode in popData.FIPSCode.unique() for month in range(1,7)])
otherFIPS = pd.Series([FIPSCode for FIPSCode in popData.FIPSCode.unique() for year in range(1991,2021) for month in range(1,13)])

monthlyPop = popData[popData.year<2021].apply(lambda x: np.linspace(x['pop'],
                                                                    x.popLead,13)[:12],
                                              axis=1).apply(pd.Series).stack().reset_index(drop = True)

In [31]:
popDataMonthly = pd.DataFrame({'FIPSCode':FIPS1990,'year':years1990,'month':months1990})
popDataMonthly = pd.concat([popDataMonthly,pd.DataFrame({'FIPSCode':otherFIPS,'year':otherYears,'month':otherMonths})])
popDataMonthly = pd.concat([popDataMonthly,pd.DataFrame({'FIPSCode':FIPS2021,'year':years2021,'month':months2021})])
popDataMonthly = popDataMonthly.sort_values(by=['FIPSCode','year','month'])
popDataMonthly = popDataMonthly.reset_index(drop=True)

In [32]:
popDataMonthly['pop'] = monthlyPop

In [33]:
popDataJuly2021 = popData.loc[popData.year==2021,['FIPSCode','year','pop']].reset_index(drop=True)
popDataJuly2021['month'] = 7

In [34]:
popDataMonthly = pd.concat([popDataMonthly,popDataJuly2021])
popDataMonthly = popDataMonthly.sort_values(by=['FIPSCode','year','month'])
popDataMonthly = popDataMonthly.reset_index(drop=True)

In [35]:
months2021 = pd.Series([month for FIPSCode in popData.FIPSCode.unique() for month in range(7,13)])
years2021 = pd.Series([2021 for FIPSCode in popData.FIPSCode.unique() for month in range(7,13)])
FIPS2021 = pd.Series([FIPSCode for FIPSCode in popData.FIPSCode.unique() for month in range(7,13)])

monthlyPop2021 = [pop for pop in popDataJuly2021['pop'] for month in range(7,13)]

In [36]:
popDataMonthly2021 = pd.DataFrame({'FIPSCode':FIPS2021,'year':years2021,'month':months2021,'pop':monthlyPop2021})
popDataMonthly2021 = popDataMonthly2021.sort_values(by=['FIPSCode','year','month'])
popDataMonthly2021 = popDataMonthly2021.reset_index(drop=True)

In [37]:
popDataMonthly = pd.concat([popDataMonthly,popDataMonthly2021])
popDataMonthly = popDataMonthly.sort_values(by=['FIPSCode','year','month'])
popDataMonthly = popDataMonthly.reset_index(drop=True)

In [38]:
popDataMonthly.to_feather(os.path.join(outDir,'popDataMonthly.feather'))